In [1]:
from petpal.preproc import image_operations_4d

# PET Preprocessing with PPM

This is a brief tutorial on how to run preprocessing on PET data using the PPM software. Preprocessing includes motion correction, registration, regional TAC extraction, and other tools.

The first step is identifying the input data to be used. This tutorial will use raw PET data, anatomical data in the form of an MPRAGE, segmentation from FreeSurfer in the same space as the MPRAGE, a color table defining regions to extract TACs from (see the 'shared' folder for an example), and the radioisotope half life in seconds.

In [2]:
pet = '/export/scratch1/test-PPM/source_data/sub-001_pet-f18.nii.gz'
anat = '/export/scratch1/test-PPM/source_data/sub-001_anat.nii.gz'
seg = '/export/scratch1/test-PPM/source_data/sub-001_seg.nii.gz'
ctab = '/export/scratch1/test-PPM/source_data/gtmseg_ctab.json'
half_life = 6586.26

Next, we'll identify where we want to save processed results.

In [3]:
sum_image = '/export/scratch1/test-PPM/analysis/sub-001_pet-sum.nii.gz'
moco = '/export/scratch1/test-PPM/analysis/sub-001_pet-moco.nii.gz'
reg = '/export/scratch1/test-PPM/analysis/sub-001_pet-reg.nii.gz'
seg_resampled = '/export/scratch1/test-PPM/analysis/sub-001_seg-res.nii.gz'
tacs = '/export/scratch1/test-PPM/analysis/sub-001_tacs'

The first method we'll use is `weighted_series_sum`, which computes a PET image averaged over time, weighted by frame length: 

In [4]:
image_operations_4d.weighted_series_sum(
    input_image_4d_path=pet,
    out_image_path=sum_image,
    half_life=half_life,
    verbose=True
)

This can be used as a reference for motion correction, which is performed next. If desired, any other 3D NIfTI image file can be used as a reference, such as last frame, the average of the first 10 frames, etc.

In [5]:
image_operations_4d.motion_correction(
    input_image_4d_path=pet,
    reference_image_path=sum_image,
    out_image_path=moco,
    verbose=True
)

Now we'll want to register the motion corrected PET image to MPRAGE space. If we instead wish to use the raw PET image, and do the registration without motion correction, we would substitute `moco` with `pet` for the `input_reg_image_path` option. Likewise, the `input_calc_image_path` option can be set to any 3D image in PET space that acts as a reference for the PET data for the purpose of computing the registration.

In [6]:
image_operations_4d.register_pet(
    input_calc_image_path=sum_image,
    input_reg_image_path=moco,
    reference_image_path=anat,
    out_image_path=reg,
    verbose=True
)

With the PET registered to anatomical space, we can extract TACs for each region in the segmentation file. `register_pet` will resample the PET data to the resolution of the anatomical image, and thus also the segmentation image, however in case the segmentation data should be downsampled to the PET resolution, method `resample_segmentation` can do just this:

In [ ]:
image_operations_4d.resample_segmentation(
    input_image_4d_path=reg,
    segmentation_image_path=seg,
    out_seg_path=seg_resampled,
    verbose=True
)

Finally, we can run `write_tacs` to extract TACs from regions in the segmentation image. The regions that are written to file are taken from the file referenced in the `color_table_path`, which is a .json file listing region indices in the segmentation image and their corresponding region names. The written TAC files record the region name when written to file.

In [9]:
image_operations_4d.write_tacs(
    input_image_4d_path=reg,
    color_table_path=ctab,
    segmentation_image_path=seg,
    out_tac_path=tacs,
    verbose=True
)